# Braille Detection PyTorch

## 1. 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
import sys
from datetime import datetime

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. 필요 라이브러리 설치

In [2]:
!pip install -r "/content/drive/MyDrive/PyTorch/requirements.txt"
!pip install albumentations==0.4.6

In [3]:
import json
import collections
import pandas as pd
import numpy as np
import functools
import matplotlib.pyplot as plt
import cv2

from sklearn import preprocessing 
from tqdm import tqdm

import xml.etree.ElementTree as ET

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
import torchvision

from torchvision import transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch import nn
from torch import optim

import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset
from torch.utils.data import random_split
from torch.utils.data import SequentialSampler
from torch.utils.tensorboard import SummaryWriter

## 3. Data 확인 및 정리

In [25]:
BASE_PATH = "/content/drive/MyDrive/PyTorch/AngelinaBook"
TRAIN_PATH = os.path.join(BASE_PATH, "train")
VAL_PATH = os.path.join(BASE_PATH, "val")
TRAIN_JSON_PATH = os.path.join(TRAIN_PATH, "json")
TRAIN_IMG_PATH = os.path.join(TRAIN_PATH, "image")
TRAIN_JSON_FILES = [os.path.join(TRAIN_JSON_PATH, f) for f in os.listdir(TRAIN_JSON_PATH)]
VAL_JSON_PATH = os.path.join(VAL_PATH, "json")
VAL_IMG_PATH = os.path.join(VAL_PATH, "image")
VAL_JSON_FILES = [os.path.join(VAL_JSON_PATH, f) for f in os.listdir(VAL_JSON_PATH)]

In [7]:
print(TRAIN_JSON_FILES[:5])

['/content/drive/MyDrive/PyTorch/AngelinaBook/train/json/IMG_20190715_122826.labeled.json', '/content/drive/MyDrive/PyTorch/AngelinaBook/train/json/IMG_20190715_113026.labeled.json', '/content/drive/MyDrive/PyTorch/AngelinaBook/train/json/IMG_20190715_113048.labeled.json', '/content/drive/MyDrive/PyTorch/AngelinaBook/train/json/IMG_20190715_123331.labeled.json', '/content/drive/MyDrive/PyTorch/AngelinaBook/train/json/IMG_20190715_121640.labeled.json']


In [54]:
# json 데이터에서 imagePath, shapes 안의 label, boxes(points) 데이터를 뽑아야 한다.
def json_files_to_df(JSON_FILES, IMAGE_PATH):
  name_list = []
  box_list = []
  image_id_list = []
  json_path_list = []
  img_path_list = []
  for jPath in JSON_FILES:
    with open(jPath, encoding= 'cp1251') as jFile:
      try:
        json_data = json.load(jFile)
        image_id = json_data['imagePath']
        json_path = jPath
        img_path = os.path.join(IMAGE_PATH, image_id)
        for s in json_data['shapes']:
          name = s['label']
          box = s['points'][0] + s['points'][1]
      
          name_list.append(name)
          box_list.append(box)
          image_id_list.append(image_id)
          json_path_list.append(json_path)
          img_path_list.append(img_path)
      except:
        print(repr(jFile))
        print(sys.exc_info())
  
  a = {
       "image_id": image_id_list,
       "names": name_list,
       "boxes": box_list,
       "json_path": json_path_list,
       "img_path": img_path_list
  }
  df = pd.DataFrame.from_dict(a)
  # df = pd.DataFrame(dict([(k,pd.Series(v)) for k, v in a.items()]))
  # df = df.transpose()

  return df


df = json_files_to_df(TRAIN_JSON_FILES, TRAIN_IMG_PATH)

In [55]:
df.head()

,image_id,names,boxes,json_path,img_path
0,IMG_20190715_122826.labeled.jpg,-,"[162.19696076062263, 105.8727105588329, 182.36...",/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,/content/drive/MyDrive/PyTorch/AngelinaBook/tr...
1,IMG_20190715_122826.labeled.jpg,с,"[186.20067342122397, 100.93847013503044, 206.3...",/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,/content/drive/MyDrive/PyTorch/AngelinaBook/tr...
2,IMG_20190715_122826.labeled.jpg,я,"[210.1317342122396, 97.26046125121269, 230.546...",/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,/content/drive/MyDrive/PyTorch/AngelinaBook/tr...
3,IMG_20190715_122826.labeled.jpg,д,"[233.5458730061849, 93.66611480712889, 253.960...",/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,/content/drive/MyDrive/PyTorch/AngelinaBook/tr...
4,IMG_20190715_122826.labeled.jpg,ь,"[257.8309682210286, 89.93806628064274, 278.245...",/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,/content/drive/MyDrive/PyTorch/AngelinaBook/tr...


In [56]:
df['names'].value_counts()

о     4856
а     4261
е     3672
и     3160
т     2949
      ... 
~3       1
d        1
w        1
k        1
z        1
Name: names, Length: 83, dtype: int64

In [57]:
df['img_id'] = df['image_id'].apply(lambda x:x.split('.')).map(lambda x:x[0])
df.drop(columns=['image_id'], inplace= True)
df.head()

,names,boxes,json_path,img_path,img_id
0,-,"[162.19696076062263, 105.8727105588329, 182.36...",/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,IMG_20190715_122826
1,с,"[186.20067342122397, 100.93847013503044, 206.3...",/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,IMG_20190715_122826
2,я,"[210.1317342122396, 97.26046125121269, 230.546...",/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,IMG_20190715_122826
3,д,"[233.5458730061849, 93.66611480712889, 253.960...",/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,IMG_20190715_122826
4,ь,"[257.8309682210286, 89.93806628064274, 278.245...",/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,IMG_20190715_122826


In [58]:
enc = preprocessing.LabelEncoder()
df['labels'] = enc.fit_transform(df['names'])
df['labels'] = np.stack(df['labels'][i]+1 for i in range(len(df['labels'])))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


In [59]:
classes = df[['names','labels']].value_counts()

In [60]:
classes

names  labels
о      65        4856
а      51        4261
е      56        3672
и      59        3160
т      69        2949
                 ... 
/1     10           1
w      40           1
z      41           1
k      32           1
~3     45           1
Length: 83, dtype: int64

In [61]:
df['xmin'] = -1
df['ymin'] = -1
df['xmax'] = -1
df['ymax'] = -1

df[['xmin','ymin','xmax','ymax']]=np.stack(df['boxes'][i] for i in range(len(df['boxes'])))

df.drop(columns=['boxes'], inplace=True)
df['xmin'] = df['xmin'].astype(np.float)
df['ymin'] = df['ymin'].astype(np.float)
df['xmax'] = df['xmax'].astype(np.float)
df['ymax'] = df['ymax'].astype(np.float)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will 

In [62]:
df.drop(columns=['names'], inplace=True)
df.head()

,json_path,img_path,img_id,labels,xmin,ymin,xmax,ymax
0,/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,IMG_20190715_122826,8,162.196961,105.872711,182.366598,136.309041
1,/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,IMG_20190715_122826,68,186.200673,100.938470,206.370310,131.374800
2,/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,IMG_20190715_122826,82,210.131734,97.260461,230.546275,127.709070
3,/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,IMG_20190715_122826,55,233.545873,93.666115,253.960414,124.114723
4,/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,/content/drive/MyDrive/PyTorch/AngelinaBook/tr...,IMG_20190715_122826,79,257.830968,89.938066,278.245509,120.386675


In [63]:
len(df['img_id'].unique())

158

In [67]:
train_ids = df['img_id'].unique()
train_df = df[df['img_id'].isin(train_ids)]
print(train_df.shape)

(52301, 8)


## Test 진행중

In [64]:
class VOCDataset(Dataset):
    
    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()
        
        self.image_ids = dataframe['img_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms
    
    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        records = self.df[self.df['img_id'] == image_id]
        
        image = cv2.imread(f'{self.image_dir}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        rows, cols = image.shape[:2]
        
        boxes = records[['xmin', 'ymin', 'xmax', 'ymax']].values
        
       
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)
        
        label = records['labels'].values
        labels = torch.as_tensor(label, dtype=torch.int64)
        
        # suppose all instances are not crowd
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        # target['masks'] = None
        target['image_id'] = torch.tensor([index])
        target['area'] = area
        target['iscrowd'] = iscrowd
        
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1,0)
            
            return image, target
        
    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [68]:
def get_transform_train():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        ToTensorV2(p=1.0)
    ], bbox_params={'format':'pascal_voc', 'label_fields': ['labels']})

def get_transform_valid():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields':['labels']})

In [70]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = VOCDataset(train_df, TRAIN_IMG_PATH , get_transform_train())
# valid_dataset = VOCDataset(valid_df, IMG_PATH, get_transform_valid())


# split the dataset in train and test set
indices = torch.randperm(len(train_dataset)).tolist()


train_data_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn
)

"""valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)"""

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


'valid_data_loader = DataLoader(\n    valid_dataset,\n    batch_size=4,\n    shuffle=False,\n    num_workers=4,\n    collate_fn=collate_fn\n)'

In [71]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Test 진행중

In [72]:
images, targets= next(iter(train_data_loader))
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

plt.figure(figsize=(20,20))
for i, (image, target) in enumerate(zip(images, targets)):
    plt.subplot(2,2, i+1)
    boxes = targets[i]['boxes'].cpu().numpy().astype(np.int32)
    sample = images[i].permute(1,2,0).cpu().numpy()
    names = targets[i]['labels'].cpu().numpy().astype(np.int64)
    for i,box in enumerate(boxes):
        cv2.rectangle(sample,
                      (box[0], box[1]),
                      (box[2], box[3]),
                      (0, 0, 220), 2)
        cv2.putText(sample, classes[names[i]], (box[0],box[1]+15),cv2.FONT_HERSHEY_COMPLEX ,0.5,(0,220,0),1,cv2.LINE_AA)  

    plt.axis('off')
    plt.imshow(sample)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


error: ignored